## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-18-10-51-36 +0000,nypost,Trump makes dig at Prince Harry after praising...,https://nypost.com/2025/09/18/world-news/trump...
1,2025-09-18-10-49-58 +0000,nyt,Live Updates: Trump and Starmer to Talk Busine...,https://www.nytimes.com/live/2025/09/18/world/...
2,2025-09-18-10-49-56 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/18/world/...
3,2025-09-18-10-49-38 +0000,nyt,U.K. to Formally Recognize Palestinian Stateho...,https://www.nytimes.com/2025/09/18/world/europ...
4,2025-09-18-10-48-09 +0000,bbc,"First 'one in, one out' migrant sent back to F...",https://www.bbc.com/news/articles/ckg653r06jgo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2363/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,64
41,kirk,29
40,charlie,26
320,new,17
258,gaza,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
338,2025-09-17-13-46-17 +0000,nyt,"As Trump Arrives, Europe’s Right Claims Charli...",https://www.nytimes.com/2025/09/17/world/europ...,155
5,2025-09-18-10-45-28 +0000,nypost,Vance describes ‘tough moment’ he broke news o...,https://nypost.com/2025/09/18/us-news/vance-de...,149
228,2025-09-17-20-29-18 +0000,nypost,Trump calls UK state dinner with King Charles ...,https://nypost.com/2025/09/17/world-news/presi...,125
177,2025-09-17-23-03-00 +0000,wsj,Some military lawyers and other Defense Depart...,https://www.wsj.com/politics/national-security...,118
103,2025-09-18-03-38-00 +0000,wsj,President Trump is frustrated with Israeli Pri...,https://www.wsj.com/world/middle-east/trump-ne...,115


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
338,155,2025-09-17-13-46-17 +0000,nyt,"As Trump Arrives, Europe’s Right Claims Charli...",https://www.nytimes.com/2025/09/17/world/europ...
268,92,2025-09-17-18-21-00 +0000,wsj,Former CDC Director Susan Monarez rebutted Rob...,https://www.wsj.com/politics/policy/susan-mona...
359,83,2025-09-17-11-44-17 +0000,cbc,Israel opens new temporary route out of Gaza C...,https://www.cbc.ca/news/world/gaza-city-israel...
232,65,2025-09-17-20-12-27 +0000,bbc,Fed Reserve cuts interest rates but cautions o...,https://www.bbc.com/news/articles/c3e75y90pw0o...
229,61,2025-09-17-20-25-00 +0000,wsj,U.S. military operations against Latin America...,https://www.wsj.com/politics/national-security...
228,51,2025-09-17-20-29-18 +0000,nypost,Trump calls UK state dinner with King Charles ...,https://nypost.com/2025/09/17/world-news/presi...
103,51,2025-09-18-03-38-00 +0000,wsj,President Trump is frustrated with Israeli Pri...,https://www.wsj.com/world/middle-east/trump-ne...
141,50,2025-09-18-01-10-17 +0000,wapo,ABC takes Jimmy Kimmel off the air over remark...,https://www.washingtonpost.com/business/2025/0...
137,47,2025-09-18-01-21-00 +0000,wsj,Democrats defeated a Republican effort to cens...,https://www.wsj.com/politics/policy/democrats-...
336,45,2025-09-17-13-47-59 +0000,cbc,Bank of Canada lowers interest rate to 2.5% in...,https://www.cbc.ca/news/business/bank-of-canad...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
